# 03 — Pipeline E2E: Full 6-Node Pipeline

Run all 6 nodes of the S4/S6 pipeline on a claim and inspect inputs/outputs at each step.

**Nodes covered:**
1. **Claim Decomposer** (function) — entities, PICO, sub-claims
2. **Retrieval Planner** (ReAct agent) — method selection per sub-claim
3. **Evidence Retriever** (ReAct agent) — multi-source search, reranking, dedup
4. **Evidence Grader** (ReAct agent) — study type, methodology, relevance, GRADE
5. **Verdict Agent** (ReAct agent) — weigh evidence, reconcile conflicts, assign verdicts
6. **Safety Checker** (function) — flag dangerous health advice

Each step shows the state delta — what the node added to the pipeline state.

In [4]:
import sys
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore')

import json
import pandas as pd
from IPython.display import display, Markdown

from src.models import FactCheckState
from src.functions.decomposer import run_decomposer
from src.functions.safety_checker import run_safety_checker, SAFETY_CATEGORIES
from systems.s4_langgraph.agents.retrieval_planner import run_retrieval_planner
from systems.s4_langgraph.agents.evidence_retriever import run_evidence_retriever
from systems.s4_langgraph.agents.evidence_grader import run_evidence_grader
from systems.s4_langgraph.agents.verdict_agent import run_verdict_agent

In [5]:
# Choose a claim to test
CLAIM = "The MMR vaccine causes autism in children"

# Build initial pipeline state
state: FactCheckState = {
    "claim": CLAIM,
    "pico": None,
    "sub_claims": [],
    "entities": {},
    "retrieval_plan": {},
    "evidence": [],
    "extracted_figures": [],
    "evidence_quality": {},
    "verdict": "",
    "confidence": 0.0,
    "explanation": "",
    "safety_flags": [],
    "is_dangerous": False,
    "agent_trace": [],
    "total_cost_usd": 0.0,
    "total_duration_seconds": 0.0,
}

print(f'Claim: "{CLAIM}"')
print(f'State keys: {list(state.keys())}')

Claim: "The MMR vaccine causes autism in children"
State keys: ['claim', 'pico', 'sub_claims', 'entities', 'retrieval_plan', 'evidence', 'extracted_figures', 'evidence_quality', 'verdict', 'confidence', 'explanation', 'safety_flags', 'is_dangerous', 'agent_trace', 'total_cost_usd', 'total_duration_seconds']


---
## Step 1: Claim Decomposer (function node)

**What it does:** Extracts medical entities via scispaCy NER, extracts PICO elements, decomposes the claim into atomic sub-claims.

**Inputs:** `claim` (string)  
**Outputs:** `entities`, `pico`, `sub_claims`

In [6]:
state = await run_decomposer(state)

# --- Entities ---
print("ENTITIES")
print("=" * 50)
entities = state["entities"]
for etype, elist in entities.items():
    if elist:
        print(f"  {etype}: {elist}")
if not any(v for v in entities.values()):
    print("  (none detected)")

ENTITIES
  conditions: ['MMR', 'vaccine', 'autism', 'children']


In [7]:
# --- PICO ---
print("PICO EXTRACTION")
print("=" * 50)
pico = state["pico"]
if pico:
    pico_df = pd.DataFrame([{
        "Element": el,
        "Value": getattr(pico, el.lower()) or "(null)",
    } for el in ["Population", "Intervention", "Comparison", "Outcome"]])
    display(pico_df)
else:
    print("  (no PICO)")

PICO EXTRACTION


,Element,Value
0,Population,children
1,Intervention,MMR vaccine
2,Comparison,(null)
3,Outcome,autism


In [8]:
# --- Sub-claims ---
print("SUB-CLAIMS")
print("=" * 50)
sc_rows = []
for sc in state["sub_claims"]:
    row = {"id": sc.id, "text": sc.text}
    if sc.pico:
        row["P"] = sc.pico.population or ""
        row["I"] = sc.pico.intervention or ""
        row["C"] = sc.pico.comparison or ""
        row["O"] = sc.pico.outcome or ""
    sc_rows.append(row)

display(pd.DataFrame(sc_rows))

trace = state["agent_trace"][-1]
print(f"\nDuration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

SUB-CLAIMS


,id,text,P,I,C,O
0,sc-1,The MMR vaccine causes autism in children,children,MMR vaccine,,autism



Duration: 4.45s | Cost: $0.0020


---
## Step 2: Retrieval Planner (ReAct agent)

**What it does:** Examines each sub-claim's characteristics and decides which retrieval methods to use.

**Inputs:** `sub_claims`, `entities`, `pico`  
**Outputs:** `retrieval_plan` (dict mapping sub-claim ID → list of methods)

Uses ReAct with Claude if `ANTHROPIC_API_KEY` is set, otherwise falls back to rule-based keyword matching.

In [9]:
state = await run_retrieval_planner(state)

print("RETRIEVAL PLAN")
print("=" * 60)
plan_rows = []
for sc_id, methods in state["retrieval_plan"].items():
    sc_text = next((sc.text for sc in state["sub_claims"] if sc.id == sc_id), "?")
    plan_rows.append({
        "sub_claim": f"{sc_id}: {sc_text[:60]}",
        "methods": ", ".join(methods),
        "count": len(methods),
    })

display(pd.DataFrame(plan_rows))

trace = state["agent_trace"][-1]
mode = "ReAct (LLM)" if trace.reasoning_steps > 0 else "Rule-based"
print(f"\nMode: {mode} | Duration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

RETRIEVAL PLAN


,sub_claim,methods,count
0,sc-1: The MMR vaccine causes autism in children,"pubmed_api, semantic_scholar, cross_encoder",3



Mode: ReAct (LLM) | Duration: 17.0s | Cost: $0.0240


---
## Step 3: Evidence Retriever (ReAct agent)

**What it does:** Executes the retrieval plan — searches PubMed, Semantic Scholar, Cochrane, ClinicalTrials.gov, DrugBank. Re-ranks with cross-encoder, deduplicates, links evidence to sub-claims.

**Inputs:** `retrieval_plan`, `sub_claims`, `pico`, `entities`  
**Outputs:** `evidence` (list of Evidence objects), updated `sub_claims` (with evidence IDs)

In [10]:
state = await run_evidence_retriever(state)

evidence = state["evidence"]
print(f"EVIDENCE RETRIEVED: {len(evidence)} items")
print("=" * 60)

# Per sub-claim summary
for sc in state["sub_claims"]:
    sc_ev = [e for e in evidence if e.id in sc.evidence]
    by_source = {}
    for ev in sc_ev:
        by_source[ev.source] = by_source.get(ev.source, 0) + 1
    source_str = ", ".join(f"{s}: {c}" for s, c in sorted(by_source.items())) if by_source else "none"
    print(f"  [{sc.id}] {sc.text[:55]}")
    print(f"    {len(sc_ev)} items — {source_str}")

trace = state["agent_trace"][-1]
mode = "ReAct (LLM)" if trace.reasoning_steps > 0 else "Rule-based"
print(f"\nMode: {mode} | Duration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

Loading weights: 100%|██████████| 201/201 [00:00<00:00, 2408.59it/s, Materializing param=classifier.weight]                                      
BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-12-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


EVIDENCE RETRIEVED: 8 items
  [sc-1] The MMR vaccine causes autism in children
    8 items — pubmed: 5, semantic_scholar: 3

Mode: ReAct (LLM) | Duration: 35.01s | Cost: $0.0303


In [11]:
# Show all evidence items as a table
ev_rows = []
for ev in sorted(evidence, key=lambda e: e.quality_score, reverse=True):
    ev_rows.append({
        "id": ev.id,
        "source": ev.source,
        "study_type": ev.study_type or "unknown",
        "quality_score": round(ev.quality_score, 3),
        "title": ev.title[:65],
        "pmid": ev.pmid or "",
    })

ev_df = pd.DataFrame(ev_rows)
print(f"All evidence items (sorted by cross-encoder quality score):")
display(ev_df)

All evidence items (sorted by cross-encoder quality score):


,id,source,study_type,quality_score,title,pmid
0,ev-pm-30986133,pubmed,systematic_review,1.0,The MMR Vaccine and Autism.,30986133
1,ev-pm-25898051,pubmed,unknown,1.0,Autism occurrence by MMR vaccine status among ...,25898051
2,ev-pm-30831578,pubmed,unknown,1.0,"Measles, Mumps, Rubella Vaccination and Autism...",30831578
3,ev-s2-1a718b134e6c,semantic_scholar,unknown,1.0,The “ Wakefield ” Studies : Studies Hypothesiz...,
4,ev-pm-12421889,pubmed,unknown,1.0,"A population-based study of measles, mumps, an...",12421889
5,ev-s2-53f727d55f90,semantic_scholar,unknown,1.0,Confirmatory bias in health decisions: Evidenc...,32057491
6,ev-s2-5d9a99535640,semantic_scholar,unknown,1.0,Communicating science to the public: MMR vacci...,14604564
7,ev-pm-36110492,pubmed,systematic_review,1.0,Does Vaccination Increase the Risk of Autism S...,36110492


In [12]:
# Inspect a single evidence item in detail
if evidence:
    top_ev = sorted(evidence, key=lambda e: e.quality_score, reverse=True)[0]
    print(f"TOP EVIDENCE ITEM: {top_ev.id}")
    print("=" * 60)
    print(f"  Source:       {top_ev.source}")
    print(f"  Study type:   {top_ev.study_type}")
    print(f"  Quality:      {top_ev.quality_score:.4f}")
    print(f"  PMID:         {top_ev.pmid or '—'}")
    print(f"  Title:        {top_ev.title}")
    print(f"  URL:          {top_ev.url or '—'}")
    print(f"\n  Content (first 500 chars):")
    print(f"  {top_ev.content[:500]}")
else:
    print("No evidence retrieved.")

TOP EVIDENCE ITEM: ev-pm-30986133
  Source:       pubmed
  Study type:   systematic_review
  Quality:      1.0000
  PMID:         30986133
  Title:        The MMR Vaccine and Autism.
  URL:          https://pubmed.ncbi.nlm.nih.gov/30986133/

  Content (first 500 chars):
  Autism is a developmental disability that can cause significant social, communication, and behavioral challenges. A report published in 1998, but subsequently retracted by the journal, suggested that measles, mumps, and rubella (MMR) vaccine causes autism. However, autism is a neurodevelopmental condition that has a strong genetic component with genesis before one year of age, when MMR vaccine is typically administered. Several epidemiologic studies have not found an association between MMR vacc


---
## Step 4: Evidence Grader (ReAct agent)

**What it does:** Evaluates each evidence item for study type, methodological quality, relevance to sub-claims, and applies the GRADE framework. Produces per-evidence quality scores and per-subclaim evidence summaries.

**Inputs:** `evidence`, `sub_claims`, `extracted_figures`  
**Outputs:** `evidence_quality` dict with:
- `per_evidence`: study type, hierarchy weight, methodology score, relevance, GRADE, evidence strength
- `per_subclaim`: evidence count, avg strength, direction summary, top evidence IDs

Uses ReAct with Claude if `ANTHROPIC_API_KEY` is set, otherwise falls back to rule-based (uses existing metadata).

In [13]:
state = await run_evidence_grader(state)

eq = state["evidence_quality"]
per_ev = eq["per_evidence"]
per_sc = eq["per_subclaim"]

print(f"EVIDENCE GRADING RESULTS")
print(f"  Graded: {len(per_ev)} evidence items")
print(f"  Sub-claim summaries: {len(per_sc)}")

trace = state["agent_trace"][-1]
mode = "ReAct (LLM)" if trace.reasoning_steps > 0 else "Rule-based"
print(f"  Mode: {mode} | Duration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

ReAct loop hit max steps (15)


EVIDENCE GRADING RESULTS
  Graded: 4 evidence items
  Sub-claim summaries: 1
  Mode: ReAct (LLM) | Duration: 84.85s | Cost: $0.2533


In [14]:
# Per-evidence grading table
grade_rows = []
for ev_id, info in sorted(per_ev.items(), key=lambda x: x[1].get("evidence_strength", 0), reverse=True):
    # Find matching evidence object for title
    ev_obj = next((e for e in evidence if e.id == ev_id), None)
    title = ev_obj.title[:45] if ev_obj else "?"
    
    # Get direction for first sub-claim
    relevance = info.get("relevance", {})
    first_rel = next(iter(relevance.values()), {}) if relevance else {}
    
    grade_rows.append({
        "id": ev_id,
        "title": title,
        "study_type": info.get("study_type", "?"),
        "hierarchy_wt": info.get("hierarchy_weight", 0),
        "methodology": info.get("methodology_score", 0),
        "direction": first_rel.get("direction", "?"),
        "relevance": first_rel.get("score", 0),
        "strength": info.get("evidence_strength", 0),
    })

grade_df = pd.DataFrame(grade_rows)
print("Per-Evidence Grading (sorted by strength):")
display(grade_df)

Per-Evidence Grading (sorted by strength):


,id,title,study_type,hierarchy_wt,methodology,direction,relevance,strength
0,ev-pm-30986133,The MMR Vaccine and Autism.,systematic_review,0.9,1.0,opposes,1.0,0.96
1,ev-pm-25898051,Autism occurrence by MMR vaccine status among,cohort,0.6,1.0,opposes,1.0,0.84
2,ev-pm-30831578,"Measles, Mumps, Rubella Vaccination and Autis",cohort,0.6,1.0,opposes,1.0,0.84
3,ev-pm-12421889,"A population-based study of measles, mumps, a",cohort,0.6,1.0,opposes,1.0,0.00


In [15]:
# Per sub-claim summary
print("PER SUB-CLAIM EVIDENCE SUMMARY")
print("=" * 70)

sc_summary_rows = []
for sc_id, info in per_sc.items():
    sc_obj = next((sc for sc in state["sub_claims"] if sc.id == sc_id), None)
    sc_text = sc_obj.text[:50] if sc_obj else "?"
    ds = info.get("direction_summary", {})
    sc_summary_rows.append({
        "sub_claim": f"{sc_id}: {sc_text}",
        "evidence_count": info.get("evidence_count", 0),
        "avg_strength": round(info.get("avg_strength", 0), 3),
        "supports": ds.get("supports", 0),
        "opposes": ds.get("opposes", 0),
        "neutral": ds.get("neutral", 0),
    })

display(pd.DataFrame(sc_summary_rows))

# Top evidence per sub-claim
for sc_id, info in per_sc.items():
    top_ids = info.get("top_evidence_ids", [])
    if top_ids:
        print(f"\n  [{sc_id}] Top evidence: {', '.join(top_ids[:3])}")

PER SUB-CLAIM EVIDENCE SUMMARY


,sub_claim,evidence_count,avg_strength,supports,opposes,neutral
0,sc-1: The MMR vaccine causes autism in children,4,0.66,0,4,0



  [sc-1] Top evidence: ev-pm-30986133, ev-pm-25898051, ev-pm-30831578


In [16]:
# Inspect a single evidence item's full grading
if per_ev:
    # Pick the strongest
    best_id = max(per_ev, key=lambda k: per_ev[k].get("evidence_strength", 0))
    best = per_ev[best_id]
    ev_obj = next((e for e in evidence if e.id == best_id), None)
    
    print(f"DETAILED GRADING: {best_id}")
    print("=" * 60)
    if ev_obj:
        print(f"  Title: {ev_obj.title}")
        print(f"  Source: {ev_obj.source} | PMID: {ev_obj.pmid or '—'}")
    
    print(f"\n  Study Type: {best.get('study_type', '?')}")
    print(f"  Hierarchy Weight: {best.get('hierarchy_weight', 0)}")
    print(f"  Methodology Score: {best.get('methodology_score', 0)}")
    print(f"  Evidence Strength: {best.get('evidence_strength', 0)}")
    
    # Relevance per sub-claim
    rel = best.get("relevance", {})
    if rel:
        print(f"\n  Relevance:")
        for sc_id, r in rel.items():
            print(f"    {sc_id}: direction={r.get('direction', '?')}, score={r.get('score', 0):.2f}")
            if r.get("key_finding"):
                print(f"            finding: {r['key_finding'][:80]}")
    
    # GRADE
    grade = best.get("grade", {})
    if grade:
        print(f"\n  GRADE Framework:")
        for k, v in grade.items():
            print(f"    {k}: {v}")

DETAILED GRADING: ev-pm-30986133
  Title: The MMR Vaccine and Autism.
  Source: pubmed | PMID: 30986133

  Study Type: systematic_review
  Hierarchy Weight: 0.9
  Methodology Score: 1.0
  Evidence Strength: 0.96

  Relevance:
    sc-1: direction=opposes, score=1.00
            finding: Systematic review states that several epidemiologic studies have not found an as

  GRADE Framework:
    risk_of_bias: no_serious_concern
    inconsistency: no_serious_concern
    indirectness: no_serious_concern
    imprecision: no_serious_concern
    publication_bias: no_serious_concern
    overall_quality: high


---
## Step 5: Verdict Agent (ReAct agent)

**What it does:** Weighs evidence for/against each sub-claim using quality scores from the grader, reconciles conflicting evidence, assigns per-sub-claim verdicts, and synthesizes an overall 9-level verdict.

**Inputs:** `evidence`, `evidence_quality`, `sub_claims`  
**Outputs:** `verdict`, `confidence`, `explanation`, updated `sub_claims` (with verdicts)

Uses ReAct with Claude if `ANTHROPIC_API_KEY` is set, otherwise falls back to rule-based (direction counts + avg strength).

In [17]:
state = await run_verdict_agent(state)

print("VERDICT AGENT RESULTS")
print("=" * 60)
print(f"  Overall verdict: {state['verdict']}")
print(f"  Confidence: {state['confidence']:.2f}")

trace = state["agent_trace"][-1]
mode = "ReAct (LLM)" if trace.reasoning_steps > 0 else "Rule-based"
print(f"  Mode: {mode} | Duration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

VERDICT AGENT RESULTS
  Overall verdict: REFUTED
  Confidence: 0.95
  Mode: ReAct (LLM) | Duration: 18.27s | Cost: $0.0303


In [18]:
# Per sub-claim verdicts
print("PER SUB-CLAIM VERDICTS")
print("=" * 60)
verdict_rows = []
for sc in state["sub_claims"]:
    verdict_rows.append({
        "id": sc.id,
        "sub_claim": sc.text[:55],
        "verdict": sc.verdict or "pending",
        "confidence": round(sc.confidence, 3),
    })

display(pd.DataFrame(verdict_rows))

# Explanation
print(f"\nExplanation:")
explanation = state.get("explanation", "")
if explanation:
    # Word wrap
    import textwrap
    print(textwrap.fill(explanation, width=80, initial_indent="  ", subsequent_indent="  "))
else:
    print("  (no explanation)")

PER SUB-CLAIM VERDICTS


,id,sub_claim,verdict,confidence
0,sc-1,The MMR vaccine causes autism in children,REFUTED,0.95



Explanation:
  The claim that "The MMR vaccine causes autism in children" is directly
  contradicted by strong scientific evidence. A systematic review (ev-
  pm-30986133) with very high quality (strength=0.96) states that multiple
  epidemiologic studies have found no association between MMR vaccine and
  autism. This is supported by two large cohort studies: one analyzing US
  children (ev-pm-25898051, strength=0.84) showing no link between MMR vaccine
  and autism spectrum disorders, and a nationwide cohort study of 657,461
  children (ev-pm-30831578, strength=0.84) evaluating MMR vaccine and autism
  risk. The evidence is unanimous in opposing this claim with no credible
  supporting evidence identified. The scientific consensus based on extensive
  epidemiological research clearly refutes any causal link between MMR
  vaccination and autism.


---
## Step 6: Safety Checker (function node)

**What it does:** Scans the claim against 6 safety categories for dangerous health advice. Uses a single LLM call (no reasoning loop), with rule-based keyword matching as fallback.

**Inputs:** `claim`, `verdict`, `sub_claims`, `evidence`  
**Outputs:** `safety_flags`, `is_dangerous`

Safety categories:
- `stop_medication` — stopping prescribed medication
- `unproven_alternative` — replacing proven treatments
- `delay_care` — delaying necessary medical care
- `dangerous_dosage` — dangerous dosage recommendations
- `anti_vaccination` — anti-vaccination claims
- `vulnerable_population` — claims about vulnerable groups + bad verdict

In [19]:
state = await run_safety_checker(state)

print("SAFETY CHECKER RESULTS")
print("=" * 60)

safety_flags = state.get("safety_flags", [])
is_dangerous = state.get("is_dangerous", False)

if safety_flags:
    print(f"  Safety flags ({len(safety_flags)}):")
    for flag in safety_flags:
        desc = SAFETY_CATEGORIES.get(flag, flag)
        print(f"    - {flag}: {desc}")
else:
    print("  No safety flags triggered.")

print(f"\n  Is dangerous: {is_dangerous}")

trace = state["agent_trace"][-1]
print(f"\n  Duration: {trace.duration_seconds}s | Cost: ${trace.cost_usd:.4f}")

SAFETY CHECKER RESULTS
  Safety flags (2):
    - anti_vaccination: Anti-vaccination or anti-medical-establishment claims
    - vulnerable_population: Claims about vulnerable populations (children, pregnant, elderly)

  Is dangerous: False

  Duration: 3.48s | Cost: $0.0030


---
## Pipeline Summary

In [20]:
print("PIPELINE SUMMARY")
print("=" * 60)
print(f"  Claim: \"{state['claim']}\"")
print(f"  Sub-claims: {len(state['sub_claims'])}")
print(f"  Evidence items: {len(state['evidence'])}")

eq = state.get("evidence_quality", {})
print(f"  Graded evidence: {len(eq.get('per_evidence', {}))}")
print()
print(f"  VERDICT: {state['verdict']}")
print(f"  Confidence: {state['confidence']:.2f}")
print(f"  Dangerous: {state['is_dangerous']}")

safety_flags = state.get("safety_flags", [])
if safety_flags:
    print(f"  Safety flags: {', '.join(safety_flags)}")

print(f"\n  Total cost: ${state['total_cost_usd']:.4f}")
print(f"  Total duration: {state['total_duration_seconds']:.2f}s")

# Node trace table
trace_rows = []
for t in state["agent_trace"]:
    trace_rows.append({
        "agent": t.agent,
        "type": t.node_type,
        "duration_s": t.duration_seconds,
        "cost_usd": round(t.cost_usd, 4),
        "tools": ", ".join(t.tools_called) if t.tools_called else "—",
        "reasoning_steps": t.reasoning_steps,
    })

print("\nAgent Traces:")
display(pd.DataFrame(trace_rows))

PIPELINE SUMMARY
  Claim: "The MMR vaccine causes autism in children"
  Sub-claims: 1
  Evidence items: 8
  Graded evidence: 4

  VERDICT: REFUTED
  Confidence: 0.95
  Dangerous: False
  Safety flags: anti_vaccination, vulnerable_population

  Total cost: $0.3428
  Total duration: 163.07s

Agent Traces:


,agent,type,duration_s,cost_usd,tools,reasoning_steps
0,decomposer,function,4.45,0.0020,"extract_entities, extract_pico, decompose_claim",0
1,retrieval_planner,agent,17.00,0.0240,"check_guideline_coverage, analyze_claim_charac...",3
2,evidence_retriever,agent,35.01,0.0303,"rerank_evidence, search_pubmed, search_semanti...",4
3,evidence_grader,agent,84.85,0.2533,"apply_grade, assess_methodology, classify_stud...",15
4,verdict_agent,agent,18.27,0.0303,"assign_subclaim_verdict, weigh_evidence, synth...",3
5,safety_checker,function,3.48,0.0030,llm_safety_check,0


---
## Try Another Claim

Change the claim below and re-run all cells from Step 1 onwards.

In [21]:
# Quick-run: full pipeline in one cell
# Uncomment and modify the claim to test a different one

# CLAIM2 = "Vitamin D prevents COVID-19 infection"
# 
# state2: FactCheckState = {
#     "claim": CLAIM2, "pico": None, "sub_claims": [], "entities": {},
#     "retrieval_plan": {}, "evidence": [], "extracted_figures": [],
#     "evidence_quality": {}, "verdict": "", "confidence": 0.0,
#     "explanation": "", "safety_flags": [], "is_dangerous": False,
#     "agent_trace": [], "total_cost_usd": 0.0, "total_duration_seconds": 0.0,
# }
# 
# state2 = await run_decomposer(state2)
# state2 = await run_retrieval_planner(state2)
# state2 = await run_evidence_retriever(state2)
# state2 = await run_evidence_grader(state2)
# state2 = await run_verdict_agent(state2)
# state2 = await run_safety_checker(state2)
# 
# print(f"Claim: {state2['claim']}")
# print(f"VERDICT: {state2['verdict']} (confidence: {state2['confidence']:.2f})")
# print(f"Dangerous: {state2['is_dangerous']}")
# print(f"Safety flags: {state2['safety_flags']}")
# print(f"Cost: ${state2['total_cost_usd']:.4f}")
# print(f"Duration: {state2['total_duration_seconds']:.1f}s")